<a href="https://colab.research.google.com/github/Anmol-776/Anmol/blob/main/Codec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
%%writefile spam_classifier_app.py
import pandas as pd
import numpy as np
import string
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

import streamlit as st
import pickle
import os

nltk.download('stopwords')
from nltk.corpus import stopwords

# Load dataset
df = pd.read_csv('spam.csv', encoding='latin-1')[['v1', 'v2']]
df.columns = ['label', 'text']
df['label'] = df['label'].map({'ham': 0, 'spam': 1})  # 0 = not spam, 1 = spam

def clean_text(text):
    text = text.lower()
    text = ''.join([char for char in text if char not in string.punctuation])
    words = text.split()
    words = [word for word in words if word not in stopwords.words('english')]
    return ' '.join(words)

df['text_clean'] = df['text'].apply(clean_text)

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['text_clean'])
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = MultinomialNB()
model.fit(X_train, y_train)

# Save model and vectorizer
with open('spam_model.pkl', 'wb') as f:
    pickle.dump(model, f)
with open('vectorizer.pkl', 'wb') as f:
    pickle.dump(vectorizer, f)

if os.path.exists('spam_model.pkl') and os.path.exists('vectorizer.pkl'):
    with open('spam_model.pkl', 'rb') as f:
        model = pickle.load(f)
    with open('vectorizer.pkl', 'rb') as f:
        vectorizer = pickle.load(f)

st.title("📧 Spam Email Classifier")
st.markdown("Enter email text below and see the prediction.")

user_input = st.text_area("✍️ Email Content")

if st.button("🔍 Predict"):
    cleaned = clean_text(user_input)
    features = vectorizer.transform([cleaned])
    pred = model.predict(features)[0]
    label = "🚫 Spam" if pred == 1 else "✅ Not Spam"
    st.success(f"**Prediction:** {label}")


Writing spam_classifier_app.py


In [14]:
!pip install streamlit
!pip install pyngrok


In [21]:
!ngrok authtoken 2v1ZJB9hH1UNmHHlbGFMEDeb1tu_3Z2aHTCHvX4wUzBp8uYpt


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [22]:
from pyngrok import ngrok

ngrok.kill()  # Close any existing tunnels

public_url = ngrok.connect(8501)
print("Streamlit URL:", public_url)

!streamlit run spam_classifier_app.py &


Streamlit URL: NgrokTunnel: "https://d1e4-34-106-67-192.ngrok-free.app" -> "http://localhost:8501"



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.106.67.192:8501

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
  Stopping...
